In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/raw/sample_task_data.csv')

df

,Participant Number,iPad Number,T1 Snack,T2 Bathroom,T3 Marker,T4 Multipy,T5 Use Tool,T6 Movie,T7 Breadstick,T8 Nickname,T9 Joke,T10 Weekend,Unnamed: 12
0,1.0,1.0,C+6,F+4,F+4,C+5,C+3,C+6,C+4,C+2,C+11,F+2,NaN
1,2.0,2.0,C+7,C+7,C+5,C+6,C+9,C+6,C+5,C+3,C-2,C+7,NaN
2,3.0,3.0,F+3,F+4,F+4,C+7,F+2,F+4,C+3,C+2,F+8,C+3,NaN
3,4.0,1.0,C+3,C+6,C+5,C+6,C+3,S-1,S+1,F+2,I,C+3,NaN
4,5.0,2.0,C+4,F+4,F+5,C+6,C+6,C+6,S+1,C+4,C-5,F+3,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.drop(columns=['Unnamed: 12'])
df

,Participant Number,iPad Number,T1 Snack,T2 Bathroom,T3 Marker,T4 Multipy,T5 Use Tool,T6 Movie,T7 Breadstick,T8 Nickname,T9 Joke,T10 Weekend
0,1.0,1.0,C+6,F+4,F+4,C+5,C+3,C+6,C+4,C+2,C+11,F+2
1,2.0,2.0,C+7,C+7,C+5,C+6,C+9,C+6,C+5,C+3,C-2,C+7
2,3.0,3.0,F+3,F+4,F+4,C+7,F+2,F+4,C+3,C+2,F+8,C+3
3,4.0,1.0,C+3,C+6,C+5,C+6,C+3,S-1,S+1,F+2,I,C+3
4,5.0,2.0,C+4,F+4,F+5,C+6,C+6,C+6,S+1,C+4,C-5,F+3
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df.dropna()
df

,Participant Number,iPad Number,T1 Snack,T2 Bathroom,T3 Marker,T4 Multipy,T5 Use Tool,T6 Movie,T7 Breadstick,T8 Nickname,T9 Joke,T10 Weekend
0,1.0,1.0,C+6,F+4,F+4,C+5,C+3,C+6,C+4,C+2,C+11,F+2
1,2.0,2.0,C+7,C+7,C+5,C+6,C+9,C+6,C+5,C+3,C-2,C+7
2,3.0,3.0,F+3,F+4,F+4,C+7,F+2,F+4,C+3,C+2,F+8,C+3
3,4.0,1.0,C+3,C+6,C+5,C+6,C+3,S-1,S+1,F+2,I,C+3
4,5.0,2.0,C+4,F+4,F+5,C+6,C+6,C+6,S+1,C+4,C-5,F+3


In [5]:
labels = {"Participant Number":"p_number",
          "iPad Number":"ipad_number",
          "T1 Snack":"t1_snack",
          "T2 Bathroom":"t2_bathroom",
          "T3 Snack":"t3_snack",
          "T4 Multiply":"t4_multiply",
          "T5 Use Tool":"t5_help",
          "T6 Movie":"t6_movie",
          "T7 Breadstick":"t7_breadstick",
          "T8 Nickname":"t8_nickname",
          "T9 Joke":"t9_joke",
          "T10 Weekend":"t10_weekend"}
df = df.rename(columns=labels)
df

,p_number,ipad_number,t1_snack,t2_bathroom,T3 Marker,T4 Multipy,t5_help,t6_movie,t7_breadstick,t8_nickname,t9_joke,t10_weekend
0,1.0,1.0,C+6,F+4,F+4,C+5,C+3,C+6,C+4,C+2,C+11,F+2
1,2.0,2.0,C+7,C+7,C+5,C+6,C+9,C+6,C+5,C+3,C-2,C+7
2,3.0,3.0,F+3,F+4,F+4,C+7,F+2,F+4,C+3,C+2,F+8,C+3
3,4.0,1.0,C+3,C+6,C+5,C+6,C+3,S-1,S+1,F+2,I,C+3
4,5.0,2.0,C+4,F+4,F+5,C+6,C+6,C+6,S+1,C+4,C-5,F+3


In [7]:
df = df.astype({'p_number': 'int32'})
df = df.astype({'ipad_number': 'int32'})

df.dtypes

p_number          int32
ipad_number       int32
t1_snack         object
t2_bathroom      object
T3 Marker        object
T4 Multipy       object
t5_help          object
t6_movie         object
t7_breadstick    object
t8_nickname      object
t9_joke          object
t10_weekend      object
dtype: object

In [8]:
df

,p_number,ipad_number,t1_snack,t2_bathroom,T3 Marker,T4 Multipy,t5_help,t6_movie,t7_breadstick,t8_nickname,t9_joke,t10_weekend
0,1,1,C+6,F+4,F+4,C+5,C+3,C+6,C+4,C+2,C+11,F+2
1,2,2,C+7,C+7,C+5,C+6,C+9,C+6,C+5,C+3,C-2,C+7
2,3,3,F+3,F+4,F+4,C+7,F+2,F+4,C+3,C+2,F+8,C+3
3,4,1,C+3,C+6,C+5,C+6,C+3,S-1,S+1,F+2,I,C+3
4,5,2,C+4,F+4,F+5,C+6,C+6,C+6,S+1,C+4,C-5,F+3
